In [2]:
import numpy as np 
import pandas as pd 
import random
from keras.layers import merge
from keras.layers.recurrent import LSTM
import keras.backend as K
from keras.layers import BatchNormalization,concatenate,Flatten,Concatenate,Activation,Conv1D,Add ,MaxPooling1D,Masking,GRU
from keras.layers.core import Lambda, Dense, Activation
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam,RMSprop    
from keras.models import Model
from keras.layers import Input,Dropout,GlobalMaxPooling1D
from keras.callbacks import Callback,EarlyStopping
from keras.layers.core import *
from tensorflow.contrib import rnn 

import tensorflow as tf
from tensorflow import set_random_seed   

/data/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
batch_size=64
n_hidden =12
embDimSize=6
timeseq=32
is_attention=True

x =np.random.random((3200,32,6)).astype(np.float32)
y =np.vstack([np.zeros((1600,1)),np.ones((1600,1))]).astype(np.float32)

tf.reset_default_graph() 
data   = tf.placeholder(tf.float32, [None,timeseq, embDimSize],name='data')
label  = tf.placeholder(tf.float32, [None,1],name='target') 


gru_fw_cell=rnn.GRUCell(n_hidden,activation='tanh')
gru_bw_cell=rnn.GRUCell(n_hidden,activation='tanh')
outputs,output_states=tf.nn.bidirectional_dynamic_rnn(gru_fw_cell,gru_bw_cell,data,dtype=tf.float32)

outputs_concat = tf.concat(outputs,2)
atten_weight = Dense(1, activation='softmax')(outputs_concat) #在时间序列上添加注意力
output_attention= outputs_concat*atten_weight
output_attention_mean =tf.reduce_sum(output_attention,axis=2) 

current_hidden=output_states[0]

h_t = tf.concat([output_attention_mean ,current_hidden],axis=1)
h_t = tf.nn.tanh(h_t)
y_hat = Dense(1)(h_t)


loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=label,logits=y_hat))
solver = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss) 

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

total_batch=int(x.shape[0]/batch_size)
total_cost=0

attention_total=np.zeros((0,32,1))
for i in range(total_batch):
    x_batch=x[i*batch_size:(i+1)*batch_size]
    y_batch=y[i*batch_size:(i+1)*batch_size]
    _,current_loss,current_atten_weight,temp=sess.run([solver, loss,atten_weight,outputs_concat],  feed_dict={data:x_batch,label:y_batch}) 
    attention_total=np.vstack([attention_total,current_atten_weight])
        
    total_cost+=current_loss
    avg_cost=total_cost/(i+1)

    
#查看平均注意力
avg_attention_weight=np.mean(attention_total,axis=0)
print(avg_cost)
# print(avg_attention_weight)

/data/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1719: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.6631903326511384
